In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# --- 1. Data Loading and Preprocessing ---

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Attention
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib
import os

In [3]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/FYPDataset/merged_cloud_metrics.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Feature engineering
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['timestamp'].dt.dayofweek.isin([5, 6]).astype(int)
df['month'] = df['timestamp'].dt.month

metrics = ['EC2_CPUUtilization', 'EC2_MemoryUtilization', 'RDS_CPUUtilization', 'ECS_CPUUtilization']
for metric in metrics:
    df[f'{metric}_rolling_mean_6h'] = df[metric].rolling(window=6).mean()
    df[f'{metric}_rolling_std_6h'] = df[metric].rolling(window=6).std()

df['EC2_CPU_Memory_Ratio'] = df['EC2_CPUUtilization'] / df['EC2_MemoryUtilization'].replace(0, 1e-6)
df['RDS_Connections_Per_CPU'] = df['RDS_DatabaseConnections'] / df['RDS_CPUUtilization'].replace(0, 1e-6)

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

<ipython-input-3-4ecf3b5ab93b>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-3-4ecf3b5ab93b>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


# --- 2. Feature and Target Preparation ---

In [4]:
# Define target and feature columns
target_cols = ['EC2_CPUUtilization', 'EC2_MemoryUtilization', 'RDS_CPUUtilization',
               'RDS_FreeableMemory', 'ECS_CPUUtilization', 'ECS_MemoryUtilization']
feature_cols = [col for col in df.columns if col not in ['timestamp'] + target_cols]

# Apply log transformation to target variables (adding small constant to handle zeros)
EPSILON = 1e-6
for col in target_cols:
    df[col] = np.log1p(df[col] + EPSILON)

X = df[feature_cols].values
y = df[target_cols].values

# --- 3. Data Scaling and Sequence Creation ---

In [7]:
# Data scaling
scalers = {}
scalers['features'] = MinMaxScaler()
scalers['targets'] = MinMaxScaler()

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train_scaled = scalers['features'].fit_transform(X_train)
X_test_scaled = scalers['features'].transform(X_test)
y_train_scaled = scalers['targets'].fit_transform(y_train)
y_test_scaled = scalers['targets'].transform(y_test)

# Sequence creation
sequence_length = 24
prediction_horizon = 1

def create_sequences(X, y, seq_length, pred_horizon):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length - pred_horizon + 1):
        X_seq.append(X[i:(i + seq_length)])
        y_seq.append(y[i + seq_length:i + seq_length + pred_horizon])
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length, prediction_horizon)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, sequence_length, prediction_horizon)

# Custom MAPE metric that handles small values
class CustomMAPE(tf.keras.metrics.Metric):
    def __init__(self, name='custom_mape', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.constant(1e-6, dtype=tf.float32)
        absolute_percentage_errors = tf.abs((y_true - y_pred) / (y_true + epsilon))
        mape = tf.reduce_mean(absolute_percentage_errors) * 100

        self.total.assign_add(mape)
        self.count.assign_add(1.0)

    def result(self):
        return self.total / self.count

    def reset_state(self):
        self.total.assign(0.0)
        self.count.assign(0.0)

# --- 4. Model Building and Training ---

In [8]:
input_shape = (sequence_length, len(feature_cols))
num_targets = len(target_cols)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
    MaxPooling1D(pool_size=2),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(num_targets)
])

model.compile(optimizer='adam',
             loss='mse',
             metrics=['mae', CustomMAPE()])

# Training with callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

# Model training
history = model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
34790/34790 ━━━━━━━━━━━━━━━━━━━━ 279s 8ms/step - custom_mape: 229.8238 - loss: 0.0386 - mae: 0.1513 - val_custom_mape: 96.8217 - val_loss: 0.0412 - val_mae: 0.1712
Epoch 2/50
34790/34790 ━━━━━━━━━━━━━━━━━━━━ 273s 8ms/step - custom_mape: 188.6579 - loss: 0.0368 - mae: 0.1483 - val_custom_mape: 98.2802 - val_loss: 0.0402 - val_mae: 0.1681
Epoch 3/50
34790/34790 ━━━━━━━━━━━━━━━━━━━━ 275s 8ms/step - custom_mape: 211.4270 - loss: 0.0368 - mae: 0.1484 - val_custom_mape: 99.1590 - val_loss: 0.0395 - val_mae: 0.1652
Epoch 4/50
34790/34790 ━━━━━━━━━━━━━━━━━━━━ 276s 8ms/step - custom_mape: 197.0334 - loss: 0.0368 - mae: 0.1482 - val_custom_mape: 98.7999 - val_loss: 0.0390 - val_mae: 0.1632
Epoch 5/50
34790/34790 ━━━━━━━━━━━━━━━━━━━━ 276s 8ms/step - custom_mape: 222.5396 - loss: 0.0368 - mae: 0.1483 - val_custom_mape: 99.9821 - val_loss: 0.0380 - val_mae: 0.1589
Epoch 6/50
34790/34790 ━━━━━━━━━━━━━━━━━━━━ 277s 8ms/step - custom_mape: 249.5216 - loss: 0.0368 - mae: 0.1482 - val_custom_m

# --- 5. Model Evaluation and Saving ---

In [9]:
# Evaluation function
def evaluate_predictions(y_true, y_pred, target_cols):
    y_true_orig = np.expm1(scalers['targets'].inverse_transform(y_true))
    y_pred_orig = np.expm1(scalers['targets'].inverse_transform(y_pred))

    for i, col in enumerate(target_cols):
        mae = np.mean(np.abs(y_true_orig[:, i] - y_pred_orig[:, i]))
        mape = np.mean(np.abs((y_true_orig[:, i] - y_pred_orig[:, i]) / (y_true_orig[:, i] + EPSILON))) * 100
        print(f"\n{col}:")
        print(f"MAE: {mae:.4f}")
        print(f"MAPE: {mape:.4f}%")

# Make predictions and evaluate
y_pred = model.predict(X_test_seq)
evaluate_predictions(y_test_seq, y_pred, target_cols)

# Save model and scalers
os.makedirs('cloud_metrics_lstm_multi_output', exist_ok=True)
save_model(model, '/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_model.keras')
np.save('/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_features.npy', feature_cols)
joblib.dump(scalers['features'], '/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_features_scaler.joblib')
joblib.dump(scalers['targets'], '/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_targets_scaler.joblib')

10872/10872 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step


ValueError: Found array with dim 3. None expected <= 2.

Test Accuracy and Plot Predictions

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Load the saved model and scalers
loaded_model = load_model('/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_model.keras')
loaded_features = np.load('/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_features.npy', allow_pickle=True)
loaded_feature_scaler = joblib.load('/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_features_scaler.joblib')
loaded_target_scaler = joblib.load('/content/drive/MyDrive/FYPDataset/cloud_metrics_lstm_multi_output_targets_scaler.joblib')

# Make predictions on the test set
y_pred_scaled = loaded_model.predict(X_test_seq)
y_pred = loaded_target_scaler.inverse_transform(y_pred_scaled)
y_actual = loaded_target_scaler.inverse_transform(y_test_seq)

# Calculate evaluation metrics
mae = mean_absolute_error(y_actual, y_pred)
mape = mean_absolute_percentage_error(y_actual, y_pred)
r2 = r2_score(y_actual, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}")
print(f"R-squared (R2): {r2}")

# Plot predictions vs actual for each target variable
target_cols = ['EC2_CPUUtilization', 'EC2_MemoryUtilization', 'RDS_CPUUtilization', 'RDS_FreeableMemory', 'ECS_CPUUtilization', 'ECS_MemoryUtilization']

plt.figure(figsize=(15, 10))
for i, col in enumerate(target_cols):
    plt.subplot(3, 2, i + 1)
    plt.plot(y_actual[:, i], label='Actual', color='blue')
    plt.plot(y_pred[:, i], label='Predicted', color='red', linestyle='--')
    plt.title(f'{col} - Actual vs Predicted')
    plt.xlabel('Time Steps')
    plt.ylabel(col)
    plt.legend()
plt.tight_layout()
plt.show()

Visualize actual vs prediction plot

In [ ]:
import smtplib

def send_email():
    sender = "hwimalasooriya@gmail.com"
    receiver = "hasitha.20210424@iit.ac.lk"
    password = "uklf urne obbw irdf"  # Use an App Password for security

    subject = "Training Complete!"
    body = "Your LSTM model training in Colab has finished."

    message = f"Subject: {subject}\n\n{body}"

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender, password)
        server.sendmail(sender, receiver, message)

send_email()
